# Описание проекта
Нужно решить, где бурить новую скважину.
Шаги для выбора локации обычно такие:
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

## 1. Загрузка и подготовка данных. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from numpy.random import RandomState
import plotly.express as px

#### Данные геологоразведки трёх регионов:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

In [2]:
try:

    df_1 = pd.read_csv('/Users/admin/Desktop/Rudra PRACTICUM/Projects/my_project/Datasets/geo_data_0.csv')
    df_2 = pd.read_csv('/Users/admin/Desktop/Rudra PRACTICUM/Projects/my_project/Datasets/geo_data_1.csv')
    df_3 = pd.read_csv('/Users/admin/Desktop/Rudra PRACTICUM/Projects/my_project/Datasets/geo_data_2.csv')
    
except:
    
    df_1 = pd.read_csv('/datasets/geo_data_0.csv')
    df_2 = pd.read_csv('/datasets/geo_data_1.csv')
    df_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [6]:
df_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [7]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
df_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [9]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
df_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [11]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


### Вывод:
- #### загрузил данные
- #### DF не стал скреплять так как считаю это снизит predict модели
- #### проверил df с помощью функции info(), нужные мне фичи имеют подходящий тип данных, а так же убедился в отсутствии пустых значений

## 2. Обучение и проверка модели для каждого региона:

### 2.1. Разбиваю данные на обучающую и валидационную выборки в соотношении 75:25.


In [12]:
def sep(data, columns):
    target = data[columns]
    features = data.drop([columns, 'id'], axis=1)
    
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345
    )
    
    scaler = StandardScaler()
    scaler.fit(features_train)
    pd.options.mode.chained_assignment = None
    
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    return target, features, features_train, features_valid, target_train, target_valid

In [13]:
target_1, features_1, features_train_1, features_valid_1, target_train_1, target_valid_1 = sep(df_1, 'product')
target_2, features_2, features_train_2, features_valid_2, target_train_2, target_valid_2 = sep(df_2, 'product')
target_3, features_3, features_train_3, features_valid_3, target_train_3, target_valid_3 = sep(df_3, 'product')

### 2.2. Обучаю модель и делаю предсказания на валидационной выборке.

In [14]:
def model_LinearRegression(features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    
    result = mean_squared_error(target_valid, predictions_valid) ** 0.5
    
    predictions_valid = pd.Series(predictions_valid, index=target_valid.index)
    
    print(f'Средний запас предсказанного сырья = {predictions_valid.mean()}, RMSE = {result}')
    
    return model, predictions_valid, result

In [15]:
model_1, predictions_valid_1, result_1 = model_LinearRegression(
    features_train_1, target_train_1, features_valid_1, target_valid_1
)

Средний запас предсказанного сырья = 92.59256778438038, RMSE = 37.5794217150813


In [16]:
model_2, predictions_valid_2, result_2 = model_LinearRegression(
    features_train_2, target_train_2, features_valid_2, target_valid_2
)

Средний запас предсказанного сырья = 68.728546895446, RMSE = 0.8930992867756158


In [17]:
model_3, predictions_valid_3, result_3 = model_LinearRegression(
    features_train_3, target_train_3, features_valid_3, target_valid_3
)

Средний запас предсказанного сырья = 94.96504596800489, RMSE = 40.02970873393434


### 2.5. Проанализирую результаты.

In [ ]:
fig = px.scatter_matrix(df_2, dimensions=['f0', 'f1', 'f2', 'product'])
fig.show()
df_2.corr()

### Вывод:
- #### df_2 регион имеет наименьший показатель метрики так как его f2 признак сильно корелирует с признаком product и составляет 0.999397
- #### у остальных двух регионов выигрывает df_1 регион

## 3 Подготовка к расчёту прибыли:

### 3.1. Все ключевые значения для расчётов сохраняю в отдельных переменных.

In [19]:
entire_budget = 10e9

In [20]:
cont_wells = 200

In [21]:
revenue_unit = 450e3

In [22]:
total_mines = 500

### 3.2. Рассчитываю достаточный объём сырья для безубыточной разработки новой скважины. Сравниваю полученный объём сырья со средним запасом в каждом регионе.

In [23]:
well_income = entire_budget / cont_wells / revenue_unit

In [24]:
well_income

111.11111111111111

In [25]:
target_1.mean(), target_2.mean(), target_3.mean()

(92.50000000000001, 68.82500000000002, 95.00000000000004)

### Вывод:
- #### в среднем по регионам обем сырья в месторождении близок к показателю безубыточной добычи
- #### ближе всего находится df_3 регион

## 4. Пишу функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

In [26]:
def profitable_wells(subsample, target_valid):
    best_200 = subsample.sort_values(ascending=False).head(200)
    
    fact_200 = target_valid[best_200.index]
    
    sum_fact_200 = fact_200.sum()
    
    profit = sum_fact_200 * revenue_unit - entire_budget
    
    return best_200, fact_200, sum_fact_200, profit    

## 5. Считаю риски и прибыль для каждого региона:

### 5.1. Применяю технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [27]:
def bootstrap(predictions_valid, target_valid):
    state = RandomState(12345)
    oll_profit = []

    for i in range(1000):
        subsample = predictions_valid.sample(n=total_mines, random_state=state, replace=True)
        best_200, fact_200, sum_fact_200, profit = profitable_wells(subsample, target_valid)
        oll_profit.append(profit)
        
    return oll_profit

In [28]:
oll_profit_1 = bootstrap(predictions_valid_1, target_valid_1)
oll_profit_2 = bootstrap(predictions_valid_2, target_valid_2)
oll_profit_3 = bootstrap(predictions_valid_3, target_valid_3)

### 5.2. Нахожу среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [29]:
def metrics(oll_profit):
    oll_profit = pd.Series(oll_profit)
    mean_profit = oll_profit.mean()
    lower = oll_profit.quantile(0.025)
    upper = oll_profit.quantile(0.975)
    loss = 0
    for i in oll_profit:
        if i < 0:
            loss += 1
    risk_losses = loss / len(oll_profit)
    
    print(
        f'Средняя прибыль равна {mean_profit}, доверительный интервал 95% от {lower} до {upper}, риск убытков равен {risk_losses:.1%}'
    )
    return mean_profit, lower, upper, risk_losses

In [30]:
mean_profit_1, lower_1, upper_1, risk_losses_1 = metrics(oll_profit_1)

Средняя прибыль равна 396164984.8023711, доверительный интервал 95% от -37181596.26254606 до 842688161.1281354, риск убытков равен 6.9%


In [31]:
mean_profit_2, lower_2, upper_2, risk_losses_2 = metrics(oll_profit_2)

Средняя прибыль равна 456045105.7866609, доверительный интервал 95% от 109475721.4548029 до 791005513.2594593, риск убытков равен 1.5%


In [32]:
mean_profit_3, lower_3, upper_3, risk_losses_3 = metrics(oll_profit_3)

Средняя прибыль равна 404403866.56835675, доверительный интервал 95% от -58533441.0505024 до 841123996.7081368, риск убытков равен 7.6%


### Вывод:
- #### предлагаю для добычи нефти df_2 регион
- #### Средняя прибыль равна 456045105.7866609, доверительный интервал 95% от 109475721.4548029 до 791005513.2594593, риск убытков равен 1.5%
- #### как мы видим средняя прибыль в этом регионе составляет наибольшую сумму с наименьшим доверительным интервало, что увеличивает предсказание. А также наименьший процент риска убытков, который примеррно в 5 раз меньше чем в остальных ррегионах